<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

#  Vector Store

Let's save our embeddings using Chroma and also discover new tools available for us.

----

#### Quick Note to users, check the GitHub if you have installation issues, since pydantic, langchain, and chroma sometimes are out of sync and need specific version numbers to play nicely together. There was an error occuring in June of 2023, installing this specific version of pydantic solved it, but make sure to double check GitHub issues first.

For example: 
* https://github.com/hwchase17/langchain/issues/5113
* https://github.com/hwchase17/langchain/issues/7548
#### Also don't forget to restart your kernel!
---

In [1]:
# HERE ARE THE VERSION NUMBERS THAT WORKED FOR ME:
# CAREFUL WITH PYDANTIC, DO IT LAST SINCE CHROMA AND LANGCHAIN AUTO INSTALL IT AS A DEPENDENCY
# Use this to install specific versions numbers:
# !pip install package_name==0.3.26
import chromadb
print(chromadb.__version__)
import langchain
print(langchain.__version__)
import pydantic
print(pydantic.__version__)

0.5.4
0.2.10
2.8.2


In [2]:
import chromadb

In [9]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

### Load Document and Split (even if under context window, still recommended)

In [5]:
# Load the document and split it into chunks
loader = TextLoader("some_data/FDR_State_of_Union_1944.txt")
documents = loader.load()

In [6]:
# Split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

### Connect to OpenAI for Embeddings

In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_token = os.getenv("OPENAI_API_KEY")

In [10]:
# The variable OPENAI_API_KEY needs to bet set in the environment
embedding_function = OpenAIEmbeddings()

### Pass Embeddings and Docs into Chroma

In [13]:
# Load it into Chroma; we pass:
# - the documents
# - the embedding function or Embeddings class/object
# - the directory where the DB is persisted
db = Chroma.from_documents(docs,
                           embedding_function,
                           persist_directory='./speech_embedding_db')
# The persisted folder contains
# - a SQLite database
# - two parquet files with the embeddings and the texts
# - index files

### Save the new embeddings to disk

In [12]:
# Helpful to force a save
# WARNING: Not necessary anymore...
db.persist()

c:\Users\A200239740\AppData\Local\anaconda3\envs\llms\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


### Load Embeddings from Disk

In [14]:
# Here, we load a persisted DB, i.e.,
# we CONNECT to persisted DB
db_connection = Chroma(persist_directory='./speech_embedding_db',
                       embedding_function=embedding_function)

In [18]:
# Query the DB
# WARNING: Do not use questions, but rather statements
# because this is not a chat, but a similarity search
# based on the cosine similarit of the embeddings
new_doc = "What did FDR say about the cost of food law?"

In [21]:
# We can do it with the string or the vector of the string
# Default top k = 4
# We get as result Document objects of the chunks
# that were passed to the vector store
docs = db_connection.similarity_search(new_doc)

In [23]:
len(docs) # 4

4

In [25]:
type(docs[0]) # langchain_core.documents.base.Document

langchain_core.documents.base.Document

In [20]:
# Show the content of the first document
print(docs[0].page_content) # A continuation ...

(2) A continuation of the law for the renegotiation of war contractsâ€”which will prevent exorbitant profits and assure fair prices to the Government. For two long years I have pleaded with the Congress to take undue profits out of war.

(3) A cost of food lawâ€”which will enable the Government (a) to place a reasonable floor under the prices the farmer may expect for his production; and (b) to place a ceiling on the prices a consumer will have to pay for the food he buys. This should apply to necessities only; and will require public funds to carry out. It will cost in appropriations about one percent of the present annual cost of the war.

(4) Early reenactment of. the stabilization statute of October, 1942. This expires June 30, 1944, and if it is not extended well in advance, the country might just as well expect price chaos by summer.

(5) A national service law- which, for the duration of the war, will prevent strikes, and, with certain appropriate exceptions, will make available

## Add New Document

In [44]:
# load the document and split it into chunks
loader = TextLoader("some_data/Lincoln_State_of_Union_1862.txt")
documents = loader.load()

In [45]:
# split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

Created a chunk of size 611, which is longer than the specified 500
Created a chunk of size 539, which is longer than the specified 500
Created a chunk of size 686, which is longer than the specified 500


In [47]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function,persist_directory='./speech_embedding_db')

In [48]:
docs = db.similarity_search('slavery')

In [49]:
docs[0].page_content

'As to the second article, I think it would be impracticable to return to bondage the class of persons therein contemplated. Some of them, doubtless, in the property sense belong to loyal owners, and hence provision is made in this article for compensating such. The third article relates to the future of the freed people. It does not oblige, but merely authorizes Congress to aid in colonizing such as may consent. This ought not to be regarded as objectionable on the one hand or on the other, insomuch as it comes to nothing unless by the mutual consent of the people to be deported and the American voters, through their representatives in Congress.\n\nI can not make it better known than it already is that I strongly favor colonization; and yet I wish to say there is an objection urged against free colored persons remaining in the country which is largely imaginary, if not sometimes malicious.\n\nIt is insisted that their presence would injure and displace white labor and white laborers. 

### Collection class calls: https://docs.trychroma.com/reference/Collection

In [52]:
# help(db._collection)